In [ ]:
import ee

ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=RJqY3Cf3njvw4XltlHQkV1gfECVQEYhderNsGZE5Hcw&tc=_fw8snlNrhmkDHB4y-tbqxE2RXFRIj4xXHii-7pxCaM&cc=x2-LxYqrlHN91qjXTLrDqqjWnABBFU4pYRtDK5kFlc0

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh6ZL5P8mSZNHR0AJJoYIv1bfnjNZfk1MEfYUmzZrqC8X8JFGyuXM48

Successfully saved authorization token.


In [ ]:
import pprint
import numpy as np

In [ ]:
inputYear= 2012
inputMonth = 10
inputDay = 1

In [ ]:
geom = ee.Geometry.Polygon(
              [[[-18.472215922082324, 19.18652794149394],
          [-18.472215922082324, 13.025811442265432],
          [-6.6948721720823245, 13.025811442265432],
          [-6.6948721720823245, 19.18652794149394]]],
              None,
              False
)

aoi = ee.Feature(geom)

In [ ]:
NoiseThreshold = 250
MosquitoThreshold = 1000

In [ ]:
mod13q1 = ee.ImageCollection("MODIS/061/MOD13Q1")
myd13q1 = ee.ImageCollection("MODIS/061/MYD13Q1")
modis = mod13q1.merge(myd13q1).sort('system:time_start').filterBounds(geom)

In [ ]:
print('DataSize;', modis.size().getInfo())

DataSize; 996


In [ ]:
#DOY_list = ee.List(np.arange(0, 368, 8).tolist())
DOY_list = ee.List([0,8,16,24,32,40,48,56,64,72,80,88,96,
                        104,112,120,128,136,144,152,160,168,
                        176,184,192,200,208,216,224,232,240,
                        248,256,264,272,280,288,296,304,312,
                        320,328,336,344,352,360])

In [ ]:
#Changed the start date to 1 Jan 2003, used for long-term
#NDVI time series and avg NDVI calculation.
longTermAvgStartDate =  ee.Date('2003-01-01')
longTermAvgEndDate =  ee.Date('2021-12-31')
modisStart = longTermAvgStartDate

In [ ]:
#Change ImageCollection into List
listOfImages = modis.toList(modis.size())

In [ ]:
modislast = ee.Image(listOfImages.get(-1))

In [ ]:
modisEnd = ee.Date(ee.List(modislast.get('system:time_end')))
DateEnd = ee.Date(ee.List(modislast.get('system:time_start')))

In [ ]:
#TimeStamp
pprint.pprint(modisEnd.getInfo())
pprint.pprint(DateEnd.getInfo())

#Change the format
pprint.pprint(modisEnd.format().getInfo())
pprint.pprint(DateEnd.format().getInfo())

{'type': 'Date', 'value': 1678060800000}
{'type': 'Date', 'value': 1676678400000}
'2023-03-06T00:00:00'
'2023-02-18T00:00:00'


In [ ]:
modis4avg =  modis.filterDate(longTermAvgStartDate,longTermAvgEndDate)
ndvi4avg = modis4avg.select('NDVI')

In [ ]:
#function to add property DOY to each image
def addDOY(image):
  tmp_doy = image.date().getRelative('day', 'year')
  image = image.set('DOY', tmp_doy)
  return image


ndvi4avg = ndvi4avg.map(addDOY)

In [ ]:
print('ndvi4avg', ndvi4avg.first().getInfo())

ndvi4avg {'type': 'Image', 'bands': [{'id': 'NDVI', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [172800, 72000], 'crs': 'SR-ORG:6974', 'crs_transform': [231.65635826395825, 0, -20015109.354, 0, -231.65635826395834, 10007554.677003]}], 'version': 1646532393349115, 'id': 'MODIS/061/MOD13Q1/2003_01_01', 'properties': {'system:index': '1_2003_01_01', 'DOY': 0, 'system:time_start': 1041379200000, 'google:max_source_file_timestamp': 1628728918000, 'system:footprint': {'type': 'LinearRing', 'coordinates': [[-180, -90], [180, -90], [180, 90], [-180, 90], [-180, -90]]}, 'system:time_end': 1042761600000, 'system:asset_size': 29434634715}}


In [ ]:
print('----------output of basic data-------------')
print('modisStart', modisStart.format().getInfo());
print('modisEnd', modisEnd.format().getInfo());
print('modis for average', modis4avg.first().getInfo());
print('ndvi for average', ndvi4avg.first().getInfo());
print('modis size',  modis4avg.size().getInfo());

----------output of basic data-------------
modisStart 2003-01-01T00:00:00
modisEnd 2023-03-06T00:00:00
modis for average {'type': 'Image', 'bands': [{'id': 'NDVI', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [172800, 72000], 'crs': 'SR-ORG:6974', 'crs_transform': [231.65635826395825, 0, -20015109.354, 0, -231.65635826395834, 10007554.677003]}, {'id': 'EVI', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [172800, 72000], 'crs': 'SR-ORG:6974', 'crs_transform': [231.65635826395825, 0, -20015109.354, 0, -231.65635826395834, 10007554.677003]}, {'id': 'DetailedQA', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [172800, 72000], 'crs': 'SR-ORG:6974', 'crs_transform': [231.65635826395825, 0, -20015109.354, 0, -231.65635826395834, 10007554.677003]}, {'id': 'sur_refl_b01', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, '


Part 1:
get the means of each time phase
The average NDVI of each time phase is calulated as the average NDVI of the traget time phase and the time phase before the target time phase

In [ ]:
#function to get images for certain DOY in each year
def itertimephase(index, tempList):
  tempList = ee.List(tempList)
  tmp_NDVI_col = ndvi4avg.filter(ee.Filter.eq('DOY', index))
  tmp_DOY = tmp_NDVI_col.first().get('DOY')
  tmp_NDVI_img = tmp_NDVI_col.mean()
  tmp_NDVI_img = tmp_NDVI_img.set('DOY', tmp_DOY)
  tempList = tempList.add(tmp_NDVI_img)
  return tempList

In [ ]:
#average modis NDVI for each time phase
NDVI_avg_T = ee.List([])
NDVI_avg_T = ee.List(DOY_list.iterate(itertimephase, NDVI_avg_T))

In [ ]:
print(NDVI_avg_T.size().getInfo())
print(NDVI_avg_T.get(0).getInfo())

46
{'type': 'Image', 'bands': [{'id': 'NDVI', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -32768, 'max': 32767}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'DOY': 0}}


In [ ]:
#calculate max value of the image data and the pre image data
#example; max(image of DOY 360, image of DOY 344)

cur_img = NDVI_avg_T.filter(ee.Filter.eq('DOY', ee.Number(DOY_list.get(0)))).get(0)
pre_img = NDVI_avg_T.filter(ee.Filter.eq('DOY', ee.Number(DOY_list.get(-2)))).get(0)
#get the NDVI average image collection with two time phases average
#first element of time series '0'
averageNDVI_tmp = ee.ImageCollection.fromImages([cur_img, pre_img]).max().set('DOY',0)
NDVI_avg_X = ee.List([averageNDVI_tmp]);

#average NDVI image of time phase '1'
cur_img = NDVI_avg_T.filter(ee.Filter.eq('DOY', ee.Number(DOY_list.get(1)))).get(0)
pre_img = NDVI_avg_T.filter(ee.Filter.eq('DOY', ee.Number(DOY_list.get(-1)))).get(0)
#get the NDVI average image collection with two time phases average
#first element of time series '1'
averageNDVI_tmp = ee.ImageCollection.fromImages([cur_img, pre_img]).max().set('DOY',8)
NDVI_avg_U = NDVI_avg_X.add(averageNDVI_tmp)
#remove the first two time phase from DOY list as the images have already been added to the NDVI average image list
DOY_list_U = DOY_list.remove(0)
DOY_list_U = DOY_list_U.remove(8)

In [ ]:
def iterNDVIaverage(index, tempList):
  tempList = ee.List(tempList)
  cur_img = NDVI_avg_T.filter(ee.Filter.eq('DOY', ee.Number(index))).get(0)
  pre_img = NDVI_avg_T.filter(ee.Filter.eq('DOY', ee.Number(index).subtract(16))).get(0)
  averageNDVI_tmp = ee.ImageCollection.fromImages([cur_img, pre_img]).max().set('DOY',index)
  tempList = tempList.add(averageNDVI_tmp)

  return tempList

In [ ]:
NDVI_avg_U = ee.List(DOY_list_U.iterate(iterNDVIaverage, NDVI_avg_U))

Part 2: get the NDVI image collection for RVF calculation, NDVI after July 1 of the previous year are processed

In [ ]:
#extract modis data from 07/01 in previous year to designed date
def CondenceMaxNDVI(MODIScol, year):
  year = ee.Algorithms.If(ee.Number(year).gt(DateEnd.get('year')), DateEnd.get('year'), year)
  NDVI_use_start = ee.Date.fromYMD(ee.Number(year).subtract(1), 7, 1)
  NDVI_use_end = ee.Date.fromYMD(year, 12, 31)
  NDVI_col =  MODIScol.filterDate(NDVI_use_start,NDVI_use_end).select('NDVI')
  NDVI_List = NDVI_col.toList(NDVI_col.size())
  Num_List = ee.List.sequence(0,NDVI_col.size().subtract(1))

  #adding information(DOY, year, index)
  def iterNDVICol(image, tempList):
    tempList = ee.List(tempList)
    image = ee.Image(image)
    list_len = tempList.length()
    tmp_year = ee.Date(image.get('system:time_start')).get('year')
    tmp_doy = ee.Date(image.get('system:time_start')).getRelative('day', 'year')
    image_tmp = image.set('Year', tmp_year)
    image_tmp = image_tmp.set('DOY', tmp_doy)
    image_tmp = image_tmp.set('Index', list_len)
    tempList = tempList.add(image_tmp)
    return tempList

  NDVI_col_use = []
  NDVI_col_use = ee.List(NDVI_List.iterate(iterNDVICol, NDVI_col_use))

  #calculate max(img, pre_img) of these data from 07/01 in previous data
  def Timephase2moving(index):
    index_c = ee.Number(index)
    index_p = ee.Algorithms.If(index_c.eq(0), index_c, index_c.subtract(1))
    image_c = ee.Image(NDVI_col_use.get(index_c))
    Year_tmp = ee.Number(image_c.get('Year'))
    DOY_tmp = ee.Number(image_c.get('DOY'))
    Index_tmp = ee.Number(image_c.get('Index'))
    Start_date = ee.Date(image_c.get('system:time_start'))
    image_p = ee.Image(NDVI_col_use.get(index_p))
    tmp_col = ee.ImageCollection.fromImages([image_c, image_p])
    tmp_img = tmp_col.max()
    tmp_img = tmp_img.set('Year', Year_tmp)
    tmp_img = tmp_img.set('DOY', DOY_tmp)
    tmp_img = tmp_img.set('Index', Index_tmp)
    tmp_img = tmp_img.set('StartDate', Start_date)
    return tmp_img

  NDVI_col_use_new = Num_List.map(Timephase2moving)

  return NDVI_col_use_new

In [ ]:
NDVI_timeseries_list = CondenceMaxNDVI(modis, inputYear)

In [ ]:
NDVI_timeseries_list.get(0).getInfo()

{'type': 'Image',
 'bands': [{'id': 'NDVI',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}],
 'properties': {'StartDate': {'type': 'Date', 'value': 1309737600000},
  'Year': 2011,
  'Index': 0,
  'DOY': 184}}

Part 3: calculate anomolies with threshold

In [ ]:
#NDVItmp: NDVI List of the target year (from 07/01 in previous year)
#NDVIAvgs: Average NDVI List by DOY in each year
#threshold: threshold to mask noise in anomoly

def AnomolyCalculatioin(NDVItmp, NDVIAvgs, threshold):
  def AnomolySingle(img_tmp):
    img_tmp = ee.Image(img_tmp)
    Year_tmp = ee.Number(img_tmp.get('Year'))
    DOY_tmp = ee.Number(img_tmp.get('DOY'))
    Index_tmp = ee.Number(img_tmp.get('Index'))
    Start_date = ee.Date(img_tmp.get('StartDate'))
    img_avg = NDVIAvgs.filter(ee.Filter.eq('DOY',DOY_tmp)).get(0)
    img_diff = img_tmp.subtract(img_avg)
    img_diff = img_diff.gt(threshold).multiply(img_diff)
    img_diff = img_diff.set('Year', Year_tmp)
    img_diff = img_diff.set('DOY', DOY_tmp)
    img_diff = img_diff.set('Index', Index_tmp)
    img_diff = img_diff.set('StartDate', Start_date)
    return img_diff

  anomoly_NDVI_list = NDVItmp.map(AnomolySingle)
  return anomoly_NDVI_list

In [ ]:
NDVI_means_list = NDVI_avg_U
NDVI_anomoly_list = AnomolyCalculatioin(NDVI_timeseries_list, NDVI_means_list, NoiseThreshold)

In [ ]:
NDVI_anomoly_list.get(0).getInfo()

{'type': 'Image',
 'bands': [{'id': 'NDVI',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -65535,
    'max': 65535},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}],
 'properties': {'StartDate': {'type': 'Date', 'value': 1309737600000},
  'Year': 2011,
  'Index': 0,
  'DOY': 184}}

Part 4: Generate three-months consecutive image

In [ ]:
#year = inputYear
#month = inputMonth
#day = inputDay
#threshold = MosqutioThreshold
def calcConsecutives(anomolyList, year, month, day, threshold):
  inputDate = ee.Date.fromYMD(year, month, day)
  DateDiff = DateEnd.difference(inputDate, 'days')
  DateUse = ee.Date(ee.Algorithms.If(DateDiff.gt(0), inputDate, DateEnd))
  YearUse = ee.Number(DateUse.get('year'))
  DOYUse = ee.Number(DateUse.getRelative('day', 'year'))
  DOYTime = DOYUse.subtract(DOYUse.mod(ee.Number(8)))
  anomolyCol = ee.ImageCollection.fromImages(anomolyList)
  anomoly_c = anomolyCol.filter(ee.Filter.eq('DOY',DOYTime)).filter(ee.Filter.eq('Year',YearUse)).first()
  #getting images of all images during three months before the target time phases
  date_output = anomoly_c.get('StartDate')
  index = ee.Number(anomoly_c.get('Index'))
  anomoly_p1 = anomolyList.get(index.subtract(1))
  anomoly_p2 = anomolyList.get(index.subtract(2))
  anomoly_p3 = anomolyList.get(index.subtract(3))
  anomoly_p4 = anomolyList.get(index.subtract(4))
  anomoly_p5 = anomolyList.get(index.subtract(5))
  anomoly_p6 = anomolyList.get(index.subtract(6))
  anomoly_p7 = anomolyList.get(index.subtract(7))
  anomoly_p8 = anomolyList.get(index.subtract(8))
  anomoly_p9 = anomolyList.get(index.subtract(9))
  anomoly_p10 = anomolyList.get(index.subtract(10))
  anomoly_p11 = anomolyList.get(index.subtract(11))
  #calculate the average of the 12 images during the three months
  ThreeMohthCol = ee.ImageCollection.fromImages([anomoly_c,anomoly_p1,anomoly_p2,anomoly_p3,anomoly_p4,
                                                      anomoly_p5,anomoly_p6,anomoly_p7,anomoly_p8,anomoly_p9,
                                                      anomoly_p10,anomoly_p11])
  ConsecutivesImg = ThreeMohthCol.mean()
  #convert the consecutive image as binary map, and change band name as 'RVF-Img'.
  #if value is more than mosquitothreshold;1, else;0
  ConsecutivesImg = ConsecutivesImg.gt(threshold).toShort()
  ConsecutivesImg = ConsecutivesImg.set('system:time_start',date_output)
  ConsecutivesImg = ConsecutivesImg.rename('RVF-Img')

  return ConsecutivesImg


In [ ]:
RVFImg = calcConsecutives(NDVI_anomoly_list, inputYear, inputMonth, inputDay, MosquitoThreshold)

In [ ]:
#RVFImg.getInfo()

Part 5: display

## Way1 : Using folium to display the result

In [ ]:
# Define a method for displaying Earth Engine image tiles on a folium map.

import folium
def add_ee_layer(self, ee_object, vis_params, name):

    try:
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)

    except:
        print("Could not display {}".format(name))

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
def save_on_gdrive(image, dir_name, file_name, scale):
    task = ee.batch.Export.image.toDrive(**{
        'image': image,
        'description': file_name,
        'folder':dir_name,
        'scale': scale,
        'crs': 'EPSG:4326',
        'maxPixels': 1e13
    })
    # Run exporting
    task.start()
    print('Done.')

In [ ]:
# Define the visualization parameters.
aoiVizParams = {'min': 0,  'max': 1,'palette': 'FFFFFF', 'opacity': -1}
imageVizParams = {  'min': 0,  'max': 1,  'palette': ['FF0000','FF0000']}

# Define a map
map = folium.Map()
img_present = RVFImg.updateMask(RVFImg.neq(0))

# Add the image layer to the map and display it.
map.add_ee_layer(geom, aoiVizParams, 'area of interest')
map.add_ee_layer(img_present.clip(geom), imageVizParams, 'false color composite')

In [ ]:
import pandas as pd
data = pd.read_csv('RVFdf.csv')
def plotDot(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        radius=1,
                        weight=1,
                        color='Blue').add_to(map)

#use df.apply(,axis=1) to "iterate" through every row in your dataframe
data.apply(plotDot, axis = 1)

0       None
1       None
2       None
3       None
4       None
        ... 
1882    None
1883    None
1884    None
1885    None
1886    None
Length: 1887, dtype: object

In [ ]:
map

## Way2 : Using geemap to display the result

In [ ]:
import geemap

ModuleNotFoundError: ignored

In [ ]:
Map = geemap.Map()

In [ ]:
img_present = RVFImg.updateMask(RVFImg.neq(0))

#Map.addLayer(aoi, aoiVizParams, 'Mount Everest')
Map.addLayer(img_present.clip(aoi), imageVizParams, 'anomalies')
Map.addLayer(geom,{}, 'area of interest')

Map.centerObject(aoi, 5)

In [ ]:
Map.addLayerControl()
Map

Data export

In [ ]:
#DataExport
task = ee.batch.Export.image.toDrive(**{
  'image': img_present,
  'fileNamePrefix': 'RVFImg',
  'folder':'earthengine',
  'scale': 250,
  'region': aoi.geometry().bounds(),
  'maxPixels': 1e10
})

#task.start()
print('Done!')